In [10]:
!python3 -m pip install  pandarallel ipywidgets

  Using cached ipywidgets-7.6.5-py2.py3-none-any.whl (121 kB)
  Using cached widgetsnbextension-3.5.1-py2.py3-none-any.whl (2.2 MB)
  Using cached jupyterlab_widgets-1.0.2-py3-none-any.whl (243 kB)


In [11]:
from urllib.parse import urlparse
import http.client, sys

def check_url(url):
  url = urlparse(url)
  conn = http.client.HTTPConnection(url.netloc, timeout=10)   
  conn.request("HEAD", url.path)
  if conn.getresponse():
    return True
  else:
    return False

def get_correct_url(url):
    try:
        if check_url('https://'+url):
            return 'https://'+url
        else:
            return 'http://'+url
    except Exception as e:
        return 'http://' + url

In [12]:
import pandas as pd
from tqdm.auto import tqdm
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)
tqdm.pandas()

df = pd.read_csv('top-1m.csv', names=['rank', 'url'])
df_top_1000 = df[:1000]
df = df[1000:100000]
df_rest = df.sample(9000)
df = pd.concat([df_top_1000, df_rest])

INFO: Pandarallel will run on 64 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [16]:
df.to_csv('10k_sites.csv')

In [14]:
df['new_url'] = df.url.parallel_apply(get_correct_url)